In [ ]:
%pip install pandas

In [ ]:
import pandas as pd

# **Extração de Dados**

In [ ]:
# Dicionário com os caminhos para os arquivos.
arquivos = {
    '2021': 'data/recife-dados-receitas-2021.csv',
    '2022': 'data/recife-dados-receitas-2022.csv',
    '2023': 'data/recife-dados-receitas-2023.csv'
}

lista_dataframes = [] # Uma lista vazia para guardar os dados de cada ano temporariamente.

for ano, arquivo in arquivos.items():
    try:
        df_temp = pd.read_csv(
            arquivo,
            sep=';',           # Separador: CSVs do governo brasileiro usam ponto e vírgula.
            encoding='utf-8',  # Codificação: Para ler acentos (ç, ã, é) corretamente.
            thousands='.',     # Milhar: Avisa que o ponto serve para milhar (1.000).
            decimal=',',       # Decimal: Avisa que a vírgula é decimal (0,50).
            dtype=str          # Força a ler como texto inicialmente
        )
       
        # Cria uma coluna nova para saber de qual ano veio aquele dado.
        df_temp['ano'] = ano

        # Adicionando esse pedaço na nossa lista.
        lista_dataframes.append(df_temp)

    except Exception as e:
        # Se der erro (arquivo não existe, nome errado), recebemos uma mensagem de erro.
        print(f"Erro em {ano}: {e}")

## **Consolidação/Concatenação**

In [ ]:
# Só rodamos se a lista não estiver vazia.
if lista_dataframes:
    # pd.concat = Concatenar.
    df_raw = pd.concat(lista_dataframes, ignore_index=True)
    print(f"Total de linhas no df unificado: {len(df_raw)}")

## **Resultado e Informações**

In [ ]:
# Verificando o resultado final e as informações sobre os tipos de dados e 
display(df_raw.head(5))
display(df_raw.info())

## **Lógica de Limpeza**

In [ ]:
def limpar_moeda(valor):
        if isinstance(valor, str): # Só limpa se for texto
            valor_limpo = valor.replace('R$', '').strip()
            valor_limpo = valor_limpo.replace('.', '')    # Tira milhar
            valor_limpo = valor_limpo.replace(',', '.')   # Vírgula vira ponto
            
            try:
                return float(valor_limpo)
            except:
                return 0.0
        return valor

## **Aplicando a limpeza nas colunas de valor**

In [ ]:
cols_valores = ['receita_prevista', 'receita_prevista_acrescimo', 'receita_prevista_atualizada', 'receita_arrecadada']
for col in cols_valores:
    df_raw[col] = df_raw[col].apply(limpar_moeda)

## **Análise e Tratamento de Nulos**

In [ ]:
print("--- ANÁLISE DE VALORES NULOS POR COLUNA ---")

# Vamos mostrar a contagem de nulos por coluna
print(df_raw.isnull().sum())

# Agora, vamos verificar as 5 primeiras linhas do DataFrame que possuem a coluna 'rubrica_receita_nome' nula
coluna_teste = 'rubrica_receita_nome'

if coluna_teste in df_raw.columns:
    print(f"Mostrando as primeiras 5 linhas onde '{coluna_teste}' está nula:")
    
    # O comando .isnull() retorna um boolean. Usamos isso para filtrar.
    linhas_com_problema = df_raw[df_raw[coluna_teste].isnull()]
    
    if not linhas_com_problema.empty:
        display(linhas_com_problema.head())
    else:
        print(f"Nenhum valor nulo encontrado na coluna {coluna_teste}!")

coluna_categoria = 'categoria_receita_nome'
if coluna_categoria in df_raw.columns:
    print(f"Valores únicos em {coluna_categoria} (Amostra):")
    print(df_raw[coluna_categoria].unique()[:20]) # Mostra os 20 primeiros tipos encontrados